In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
X = np.load('sequences.npy')
labels = np.load('labels.npy')

In [3]:
y = to_categorical(labels).astype(int)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

Neural Network

In [5]:
signs = np.array(['hello', 'please', 'thanks', 'receipt', 'more', 'price', 'order', 'wait', 'bag', 'water','0', '1', '2', '3', '4', '5', '(6W)', '7', '8', '(9F)', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z'])

In [6]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=30, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
# Save the model temporarily if needed
# model.save("gesture_model.keras")
# model = tf.keras.models.load_model("gesture_model.keras")  # if using saved version

# Create TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set supported ops
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

converter.allow_custom_ops = True

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("gesture_classifier.tflite", "wb") as f:
    f.write(tflite_model)